In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tracts = gpd.read_file('censustract.geojson')[['name','geometry']]

In [8]:
tracts

,name,geometry
0,52,"[-74.092646 40.711658, -74.09227300000001 40...."
1,24,"[-74.04724400000001 40.724928, -74.0471650000..."
2,56,"[-74.095862 40.708725, -74.095321 40.709228, ..."
3,9.02,"[-74.066463 40.736345, -74.06640899999999 40...."
4,43,"[-74.075661 40.715687, -74.07482899999999 40...."
...,...,...
62,72,"[-74.053901 40.727592, -74.053855 40.727714, ..."
63,73,"[-74.056088 40.717345, -74.055685 40.717302, ..."
64,23,"[-74.04937200000001 40.722224, -74.04937 40.7..."
65,10,"[-74.06221600000001 40.74581, -74.062066 40.7..."


In [3]:
tracts['geometry'] = tracts['geometry'].astype(str)

In [4]:
tracts['geometry'] = tracts['geometry'].str[16:-3]

In [5]:
tracts['geometry'] = tracts['geometry'].astype(str)

In [6]:
tracts['geometry'] = tracts['geometry'].str.split(',')

In [7]:
tracts['geometry'].loc[0][1].split(' ')[1:]

['-74.09227300000001', '40.711896']

In [8]:
def centerpoints_lng(x):
    target_ = x.copy()
    for i in range(len(x)):
        if i == 0:
            target_[i] = target_[i].split(' ')
        else:
            target_[i] = target_[i].split(' ')[1:]
    target_ = np.array(target_)
    for i in range(len(target_)):
        try:
            target_[i] = np.array(target_[i]).astype(float)
        except:
            target_[i] = np.array([np.nan,np.nan])
    target_ = target_[:,0].astype(float)
    center_lng = (max(target_) + min(target_))/2
    return center_lng

In [9]:
def centerpoints_lat(x):
    target_ = x.copy()
    for i in range(len(x)):
        if i == 0:
            target_[i] = target_[i].split(' ')
        else:
            target_[i] = target_[i].split(' ')[1:]
    target_ = np.array(target_)
    for i in range(len(target_)):
        try:
            target_[i] = np.array(target_[i]).astype(float)
        except:
            target_[i] = np.array([np.nan,np.nan])
    target_ = target_[:,1].astype(float)
    center_lng = (max(target_) + min(target_))/2
    return center_lng

In [10]:
tracts['center_lng'] = tracts['geometry'].apply(lambda x:centerpoints_lng(x))

In [11]:
tracts['center_lat'] = tracts['geometry'].apply(lambda x:centerpoints_lat(x))

In [13]:
tracts = tracts.drop('geometry',axis=1)

In [18]:
tracts = pd.DataFrame(tracts)

In [26]:
tracts = tracts.set_index('name')

In [27]:
tracts

,center_lng,center_lat
name,,
52,-74.085396,40.708846
24,-74.045277,40.726574
56,-74.091518,40.705998
9.02,-74.062474,40.741328
43,-74.071393,40.716794
...,...,...
72,-74.047732,40.728519
73,-74.047275,40.713518
23,-74.047693,40.723929


In [28]:
tracts.to_json('centerpoints.json',orient='index')